### Test Instructions
The purpose of this simple coding test is to allow us to get a grasp on the quality of your code and to identify the role that would suit you best within our team. You don't NEED to be able to answer all of the items, but the ones that you do answer need to be correct. Feel free to use Google or any other tools that you prefer to complete these tasks.

Install any needed third-party libraries below this block. Please install the minimum amount of libraries you need.

!pip install whatever_you_need

All the required libraries are also mentioned under the specific tasks which need them.

In [ ]:
!pip install discord
!pip install nest_asyncio
!pip install langchain
!pip install openai

### Task 1 (Data manipulation / Visualization)
Download the file: https://drive.google.com/file/d/1R_M3xI3b_BqAN_xWLqJDneg273X_sok5/view?usp=sharing

In `message_data.csv` there's two columns, one corresponds to the unique id of someone sending a message and the other one to when they sent it.

Generate an interactive chart that shows the percentage of daily messages sent by new users (those who sent their first message ever on a given day).

Please upload the `message_data.csv` file in the Colab session for the following code cell to work.

In [ ]:
import pandas as pd
import plotly.express as px

# Load the data from the CSV file
df = pd.read_csv('message_data.csv')

# Convert the 'message_time' column to datetime format and extract Date
df['message_time'] = pd.to_datetime(df['message_time'])
df['Date'] = pd.to_datetime(df['message_time']).dt.date
df = df.drop(columns=['message_time'])
# Sort the data by 'Date'
df = df.sort_values(by='Date')
df.head()
# Group the data by user and get the minimum 'Date' for each user
first_message_dates = df.groupby(['author_id', 'Date'])['message_id'].agg('count').reset_index()
first_message_dates = first_message_dates.drop_duplicates('author_id')
first_message_dates = first_message_dates.groupby('Date')['message_id'].agg('sum').reset_index()
first_message_dates.columns = ['Date', 'New User Daily Messages']

# Getting the total count of messages sent per day
all_message_dates = df.groupby('Date')['message_id'].count().reset_index()
all_message_dates.columns = ['Date', 'Total Daily Messages']

# Merge the 2 dataframes
daily_data = pd.merge(first_message_dates, all_message_dates, on='Date')
daily_data['Percentage New Users'] = (daily_data['New User Daily Messages'] / daily_data['Total Daily Messages']) * 100

# Plot the interactive chart
fig = px.line(daily_data, x='Date', y='Percentage New Users', title='Percentage of Daily Messages Sent by New Users')
fig.show()


### Task 2 (Django)
**To take the django portion of this test go to: https://github.com/llu13701/cm1_python_test and follow the instructions.**

Document with screenshots: https://docs.google.com/document/d/11Snk-s_14LN3uKWchOB0L_MP9UZ4buWB9mO1md4zt1E/edit?usp=sharing


YouTube video: https://youtu.be/y93_cxb682o

### Task 3 (LangChain)
Write a simple Chain that does the following:
- Uses openai's gpt-3.5-turbo model.
- Remembers the last 2 message exchanges, as well as the system message.
- Greets people and tells a joke about their name.

Assumptions:
1. The user sends the first message telling the system their name (so that the model can greet using the name).
2. Then, the system message of greeting is generated, followed by a casual conversation, while retaining memory of the last 2 exchanges.

In [ ]:
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY="YOUR_OPENAI_API_KEY" #should be loaded from env

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
# from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are starting a conversation. You tell people a joke about their name."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferWindowMemory(k=2, return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)
conversation.predict(input="Hi there, I'm Jane!")




> Entering new ConversationChain chain...
Prompt after formatting:
System: You are starting a conversation. You tell people a joke about their name.
Human: Hi there, I'm Jane!

> Finished chain.


"Hi Jane! Nice to meet you. Did you know that there's a joke about your name? Why did Jane bring a ladder to the bar? Because she heard the drinks were on the house! 😄"

In [ ]:
conversation.predict(input="What's the weather like today?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are starting a conversation. You tell people a joke about their name.
Human: Hi there, I'm Jane!
AI: Hi Jane! Nice to meet you. Did you know that there's a joke about your name? Why did Jane bring a ladder to the bar? Because she heard the drinks were on the house! 😄
Human: What's the weather like today?

> Finished chain.


"Haha, sorry if my joke didn't land. As an AI, I don't have access to real-time weather information. But I can try to help you find out the weather in your area if you'd like! Just let me know where you're located."

In [ ]:
conversation.predict(input="Which planets are in the solar system?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are starting a conversation. You tell people a joke about their name.
Human: Hi there, I'm Jane!
AI: Hi Jane! Nice to meet you. Did you know that there's a joke about your name? Why did Jane bring a ladder to the bar? Because she heard the drinks were on the house! 😄
Human: What's the weather like today?
AI: Haha, sorry if my joke didn't land. As an AI, I don't have access to real-time weather information. But I can try to help you find out the weather in your area if you'd like! Just let me know where you're located.
Human: Which planets are in the solar system?

> Finished chain.


"Ah, I see you're changing the topic. That's alright! In our solar system, we have eight planets: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto used to be considered the ninth planet, but it was reclassified as a dwarf planet in 2006. Fascinating, isn't it? Is there anything specific you'd like to know about the planets?"

In [ ]:
#only retains memory of the last 2 exchanges (as given by k=2 in ConversationBufferWindowMemory); we can see that the model has forgotten the first greeting and the name Jane.
conversation.predict(input="What's the color of water?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are starting a conversation. You tell people a joke about their name.
Human: What's the weather like today?
AI: Haha, sorry if my joke didn't land. As an AI, I don't have access to real-time weather information. But I can try to help you find out the weather in your area if you'd like! Just let me know where you're located.
Human: Which planets are in the solar system?
AI: Ah, I see you're changing the topic. That's alright! In our solar system, we have eight planets: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto used to be considered the ninth planet, but it was reclassified as a dwarf planet in 2006. Fascinating, isn't it? Is there anything specific you'd like to know about the planets?
Human: What's the color of water?

> Finished chain.


"Water is actually colorless! It may appear to have a color due to reflections or impurities, but pure water itself is transparent. So, the color you see when looking at water is often influenced by factors like the presence of minerals, suspended particles, or even the reflection of the sky. Isn't that interesting?"

### Task 4 (Discord API)

Write a very simple discord bot that says "Hello {user_name}!" to every new incoming message in a discord server.


Assumption: A 'new' incoming message is technically every incoming message on the channel that is not sent by the bot. We can also optionally greet the user on only the first message from a user by triggering a response from the bot in the 'on_member_join' event (instead of from the 'on_message' event as in this task) when the user sends a message.

In [ ]:
!pip install discord
!pip install nest_asyncio

In [ ]:
import discord

import nest_asyncio
nest_asyncio.apply()

# Discord Token; should be defined in a .env file
TOKEN = 'YOUR_DISCORD_TOKEN'

intents = discord.Intents.all()

client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print('Bot is ready.')

@client.event
async def on_message(message):
    print("A new message has been sent: ", message)
    if message.author == client.user:
        return
    response = f"Hello {message.author.name}!"
    await message.channel.send(response)

client.run(TOKEN)

2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
2023-07-17 03:08:30 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-07-17 03:08:32 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 618c1b29355370530d4e8a2398ecac87).
2023-07-17 03:08:32 INFO     discord.gateway Shard ID None has connected to Ga

Bot is ready.
A new message has been sent:  <Message id=1130335293099233300 channel=<TextChannel id=1128833343291674667 name='general' position=0 nsfw=False news=False category_id=1128833343291674664> type=<MessageType.default: 0> author=<Member id=1129859993290686614 name='gbela' global_name=None bot=False nick=None guild=<Guild id=1128833342620569630 name='cm1_gayatri_belapurkar' shard_id=0 chunked=True member_count=5>> flags=<MessageFlags value=0>>
A new message has been sent:  <Message id=1130335294172967034 channel=<TextChannel id=1128833343291674667 name='general' position=0 nsfw=False news=False category_id=1128833343291674664> type=<MessageType.default: 0> author=<Member id=1128833868833759262 name='GreetBot' global_name=None bot=True nick=None guild=<Guild id=1128833342620569630 name='cm1_gayatri_belapurkar' shard_id=0 chunked=True member_count=5>> flags=<MessageFlags value=0>>


### Task 5
Use LangChain to write a discord bot that will:
- Greet a user, making a joke about their name and then proceed to answer any question they have, whenever a user sends a message.
- Use async with langchain chain calls.


In [ ]:
!pip install discord
!pip install nest_asyncio
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
import asyncio

# should be in the env; here for convenience and clarity
OPENAI_API_KEY ="YOUR_OPENAI_API_KEY"

class ConversationalBot:
  def __init__(self):

    self.prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("You answer any question that the human may have."),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}")
    ])

    self.llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0)
    self.memory = ConversationBufferWindowMemory(k=2, return_messages=True) #memory of 2 exchanges, similar to Task 3
    self.conversation = ConversationChain(memory=self.memory, prompt=self.prompt, llm=self.llm, verbose=True)


  async def generate_response(self, content):
    # using async for LangChain calls
    response = await self.conversation.apredict(input=content)
    return response


In [ ]:
import discord

import nest_asyncio
nest_asyncio.apply()

# Discord Token; should be defined in the env; here only for convenience and clarity
TOKEN = 'YOUR_DISCORD_TOKEN'

intents = discord.Intents.all()

client = discord.Client(intents=intents)
conversation_bot = ConversationalBot()

@client.event
async def on_ready():
    print('Bot is ready.')

@client.event
async def on_message(message):
    if message.author == client.user:
        return
    name = message.author.name
    make_joke_cmd = 'Greet ' + name + ' and make a joke about the name.'
    response_joke = await conversation_bot.generate_response(make_joke_cmd)
    response_message = await conversation_bot.generate_response(message.clean_content)
    response = response_joke + "\n\n" + response_message
    await message.channel.send(response)

client.run(TOKEN)

2023-07-17 03:20:23 INFO     discord.client logging in using static token
2023-07-17 03:20:23 INFO     discord.client logging in using static token
2023-07-17 03:20:23 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-07-17 03:20:23 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 6d3a040775287671e129522015d9d00c).
2023-07-17 03:20:23 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 6d3a040775287671e129522015d9d00c).
2023-07-17 03:20:23 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 6d3a040775287671e129522015d9d00c).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 6d3a040775287671e129522015d9d00c).


Bot is ready.


> Entering new ConversationChain chain...
Prompt after formatting:
System: You answer any question that the human may have.
Human: Greet gbela and make a joke about the name.

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
System: You answer any question that the human may have.
Human: Greet gbela and make a joke about the name.
AI: Hello gbela! How about a little joke about your name? 

Why did the computer go to the doctor?

Because it had a bad case of "g-bela" virus!
Human: What is a solar system?

> Finished chain.
